# Bologna TSP

This uses the code from [these slides from Bologna Business School](http://cs.unibo.it/~tong.liu3/mzn/slides_mzn.pdf).

In [3]:
 %load_ext iminizinc

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [4]:
import pandas as pd
from geopy.distance import great_circle

In [5]:
cities = pd.read_csv("data/city.csv")

In [6]:
cities

,City,Latitude,Longitude
0,Montgomery,32.361538,-86.279118
1,Phoenix,33.448457,-112.073844
2,Little Rock,34.736009,-92.331122
3,Sacramento,38.555605,-121.468926
4,Denver,39.739167,-104.984167
5,Hartford,41.767000,-72.677000
6,Dover,39.161921,-75.526755
7,Tallahassee,30.451800,-84.272770
8,Atlanta,33.760000,-84.390000
9,Boise,43.613739,-116.237651


In [9]:
cdict = cities.set_index('City').T.to_dict('list')

In [29]:
keys = list(cdict.keys())
# Can't do all 48 in a reasonable amount of time! 
keys = keys[:14] # 14 is the limit for < 1 minute run time.

# Here are values for the variables/parameters shared with MiniZinc:
n = len(keys)
dist = [[int(great_circle(tuple(cdict[key1]), tuple(cdict[key2])).miles)
           for key1 in keys]
          for key2 in keys]
city_name = keys
start_city = keys.index('Hartford') + 1 #
end_city = keys.index('Dover') + 1
print('n = ',n, '\ncity_name =', city_name, '\nstart_city = ', start_city, '\nend_city = ',end_city)

n =  14 
city_name = ['Montgomery', 'Phoenix', 'Little Rock', 'Sacramento', 'Denver', 'Hartford', 'Dover', 'Tallahassee', 'Atlanta', 'Boise', 'Springfield', 'Indianapolis', 'Des Moines', 'Topeka'] 
start_city =  6 
end_city =  7


In [30]:
%%minizinc -m bind
include "globals.mzn";

int: n;
array [1..n,1..n] of int: dist;
int: start_city;
int: end_city;

array[1..n] of var 1..n: city;
    
constraint city[1] = start_city;
constraint city[n] = end_city;
constraint all_different(city);

var int: total_distance = sum(i in 2..n)(dist[city[i-1],city[i]]);
solve minimize total_distance;

In [31]:
city

[6, 12, 11, 13, 14, 5, 10, 4, 2, 3, 1, 8, 9, 7]

In [32]:
[city_name[i-1] for i in city]

['Hartford',
 'Indianapolis',
 'Springfield',
 'Des Moines',
 'Topeka',
 'Denver',
 'Boise',
 'Sacramento',
 'Phoenix',
 'Little Rock',
 'Montgomery',
 'Tallahassee',
 'Atlanta',
 'Dover']

## Bibliography

* [GitHub site for TSP solved for 48 US States.](https://github.com/toddwschneider/shiny-salesman) But the data is in binary. 
* [Here is longitudes and latitudes](http://www.xfront.com/us_states/), then use [this](https://pypi.org/project/geopy/) to calculate the distances. 